<a href="https://colab.research.google.com/github/hrshah211/HousePriceKaggle/blob/main/Kaggle_House_Price_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing PyCaret

In [ ]:
!pip install pycaret

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 61kB 3.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


## Importing the necessary Packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from pycaret.regression import *

## Gathering the Data

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/hrshah211/HousePriceKaggle/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/hrshah211/HousePriceKaggle/main/test.csv')

# Removing the Outliers

In [ ]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)
train = train[train['SalePrice'] < 450000]
train = train[train['GarageArea'] < 1200]
train = train[train['LotFrontage'] < 154] 
train = train[train['LotArea'] < 70765] 
train = train[train['MasVnrArea'] < 1500]
train = train[train['BsmtFinSF1'] < 2000] 
train = train[train['GrLivArea'] < 4000] 

# Transforming the Target Column to remove the Skewness

In [ ]:
train["SalePrice"] = np.log(train["SalePrice"])

# Checking Null / Empty columns in Train and Test set

In [ ]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

Alley           1087
BsmtQual          31
BsmtCond          31
BsmtExposure      32
BsmtFinType1      31
BsmtFinType2      32
Electrical         1
FireplaceQu      595
GarageType        74
GarageYrBlt       74
GarageFinish      74
GarageQual        74
GarageCond        74
PoolQC          1168
Fence            946
MiscFeature     1133
dtype: int64

In [ ]:
null_columns=test.columns[test.isnull().any()]
test[null_columns].isnull().sum()

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64

# ReplaceNA method to remove the NA categorical fields to their respective reasons

In [ ]:
def ReplaceNA(dataset):

  dataset['Alley'] = dataset['Alley'].apply(lambda x : 'No Alley' if pd.isnull(x) else x)
  dataset['BsmtQual'] = dataset['BsmtQual'].apply(lambda x : 'No Basement' if pd.isnull(x) else x)
  dataset['BsmtCond'] = dataset['BsmtCond'].apply(lambda x : 'No Basement' if pd.isnull(x) else x)
  dataset['BsmtExposure'] = dataset['BsmtExposure'].apply(lambda x : 'No Basement' if pd.isnull(x) else x)
  dataset['BsmtFinType1'] = dataset['BsmtFinType1'].apply(lambda x : 'No Basement' if pd.isnull(x) else x)
  dataset['BsmtFinType2'] = dataset['BsmtFinType2'].apply(lambda x : 'No Basement' if pd.isnull(x) else x)
  dataset['FireplaceQu'] = dataset['FireplaceQu'].apply(lambda x : 'No Fireplace' if pd.isnull(x) else x)
  dataset['GarageType'] = dataset['GarageType'].apply(lambda x : 'No Garage' if pd.isnull(x) else x)
  dataset['GarageFinish'] = dataset['GarageFinish'].apply(lambda x : 'No Garage' if pd.isnull(x) else x)
  dataset['GarageQual'] = dataset['GarageQual'].apply(lambda x : 'No Garage' if pd.isnull(x) else x)
  dataset['GarageCond'] = dataset['GarageCond'].apply(lambda x : 'No Garage' if pd.isnull(x) else x)
  dataset['PoolQC'] = dataset['PoolQC'].apply(lambda x : 'No Pool' if pd.isnull(x) else x)
  dataset['Fence'] = dataset['Fence'].apply(lambda x : 'No Fence' if pd.isnull(x) else x)
  dataset['MiscFeature'] = dataset['MiscFeature'].apply(lambda x : 'None' if pd.isnull(x) else x)
  
  return dataset

# FillNA method to fill Missing Data

In [ ]:
def FillNA(dataset):

  dataset['LotFrontage'] = dataset.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
  dataset['MSZoning'] = dataset.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

  dataset['Electrical'] = dataset['Electrical'].fillna(dataset['Electrical'].mode()[0])
  dataset['KitchenQual'] = dataset['KitchenQual'].fillna(dataset['KitchenQual'].mode()[0])
  dataset['Exterior1st'] = dataset['Exterior1st'].fillna(dataset['Exterior1st'].mode()[0])
  dataset['Exterior2nd'] = dataset['Exterior2nd'].fillna(dataset['Exterior2nd'].mode()[0])
  dataset['SaleType'] = dataset['SaleType'].fillna(dataset['SaleType'].mode()[0])
  dataset['Utilities'] = dataset['Utilities'].fillna(dataset['Utilities'].mode()[0])

  dataset["Functional"] = dataset["Functional"].fillna("Typ")

  dataset['GarageYrBlt'] = dataset['GarageYrBlt'].fillna(0)
  dataset['GarageArea'] = dataset['GarageArea'].fillna(0)
  dataset['GarageCars'] = dataset['GarageCars'].fillna(0)
  dataset['MasVnrArea'] = dataset['MasVnrArea'].fillna(0)
  dataset['BsmtFinSF1'] = dataset['BsmtFinSF1'].fillna(0)
  dataset['BsmtFinSF2'] = dataset['BsmtFinSF2'].fillna(0)
  dataset['BsmtUnfSF'] = dataset['BsmtUnfSF'].fillna(0)
  dataset['TotalBsmtSF'] = dataset['TotalBsmtSF'].fillna(0)
  dataset['BsmtFullBath'] = dataset['BsmtFullBath'].fillna(0)
  dataset['BsmtHalfBath'] = dataset['BsmtHalfBath'].fillna(0)
  dataset['MasVnrType'] = dataset['MasVnrType'].fillna('None')        

  return dataset

# RemoveSkewness method to remove the Skewness in the Features

In [ ]:
def RemoveSkewness(dataset):
  numeric_feats = dataset.dtypes[dataset.dtypes != "object"].index

  skewed_feats = dataset[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
  skewness = pd.DataFrame({'Skew' :skewed_feats})
  skewness = skewness[abs(skewness) > 0.75]

  
  skewed_features = skewness.index
  lam = 0.15
  for feat in skewed_features:
    if(feat is not 'SalePrice'):
      dataset[feat] = boxcox1p(dataset[feat], lam)

  return dataset

# CreateNewColumns method to create additional Features 

In [ ]:
def CreateNewColumns(dataset):
  
  dataset['TotalArea'] = dataset['BsmtFinSF1'] + dataset['BsmtFinSF2'] + dataset['1stFlrSF'] + dataset['2ndFlrSF']
  dataset['TotalBathrooms'] = dataset['FullBath'] + (0.5*dataset['HalfBath']) + dataset['BsmtFullBath'] + (0.5*dataset['BsmtHalfBath'])
  dataset['TotalOutsideArea'] = dataset['OpenPorchSF'] + dataset['3SsnPorch'] + dataset['EnclosedPorch'] + dataset['ScreenPorch'] + dataset['WoodDeckSF']
  dataset['HasPool'] = dataset['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
  dataset['Has2Floor'] = dataset['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
  dataset['HasGarage'] = dataset['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
  dataset['HasBasement'] = dataset['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
  dataset['HasFireplace'] = dataset['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

  return dataset


# PreProcessing method to call the defined Pre-Processing Methods

In [ ]:
def PreProcessing(dataset):

  dataset = ReplaceNA(dataset)
  dataset = FillNA(dataset)
  dataset = RemoveSkewness(dataset)
  dataset = CreateNewColumns(dataset)

  return dataset

# Removing the SalePrice and Id columns

In [ ]:
trainSalePrice = train['SalePrice']
trainId = train['Id']
train = train.drop(['SalePrice','Id'], axis=1)
testId = test['Id']
test = test.drop('Id', axis=1)

# Applying PreProcessing on both the Train and Test Data

In [ ]:
train = PreProcessing(train)
test = PreProcessing(test)

# Restoring the SalePrice and Id columns

In [ ]:
train['SalePrice'] = trainSalePrice
train['Id'] = trainId
test['Id'] = testId

# Checking Null / Empty columns in Train and Test set After Pre Procssing

In [ ]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

Series([], dtype: float64)

In [ ]:
null_columns=test.columns[test.isnull().any()]
test[null_columns].isnull().sum()

Series([], dtype: float64)

# PyCaret Setup 

In [ ]:
setup = setup(train,
              session_id = 404,
              target = 'SalePrice',
              normalize  = True,
              transformation  = True,
              ignore_features = ['Id','Utilities'])

,Description,Value
0,session_id,404
1,Target,SalePrice
2,Original Data,"(1171, 89)"
3,Missing Values,False
4,Numeric Features,39
5,Categorical Features,47
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(819, 267)"


# Compare the models and selecting the Best

In [ ]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,0.0773,0.0124,0.1100,0.9226,0.0087,0.0065,0.085
ridge,Ridge Regression,0.0808,0.0131,0.1135,0.9178,0.0089,0.0068,0.026
omp,Orthogonal Matching Pursuit,0.0881,0.0152,0.1222,0.9054,0.0096,0.0074,0.026
gbr,Gradient Boosting Regressor,0.0869,0.0154,0.1234,0.9036,0.0097,0.0073,0.463
lightgbm,Light Gradient Boosting Machine,0.0922,0.0174,0.1312,0.8918,0.0103,0.0078,0.216
et,Extra Trees Regressor,0.0976,0.0195,0.1386,0.8770,0.0109,0.0082,1.595
rf,Random Forest Regressor,0.0970,0.0199,0.1400,0.8759,0.0110,0.0082,1.613
knn,K Neighbors Regressor,0.1196,0.0283,0.1675,0.8248,0.0131,0.0101,0.078
ada,AdaBoost Regressor,0.1268,0.0286,0.1686,0.8215,0.0132,0.0106,0.342
huber,Huber Regressor,0.1097,0.0309,0.1704,0.8081,0.0135,0.0092,0.168


# Creating a Ridge Model

In [ ]:
ridge = create_model('ridge')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0852,0.0160,0.1267,0.8846,0.0101,0.0073
1,0.0741,0.0091,0.0957,0.9514,0.0073,0.0062
2,0.0883,0.0138,0.1175,0.9271,0.0092,0.0074
3,0.0620,0.0063,0.0792,0.9582,0.0061,0.0052
4,0.0864,0.0144,0.1199,0.9245,0.0095,0.0073
5,0.0902,0.0165,0.1284,0.8900,0.0098,0.0075
6,0.0834,0.0173,0.1314,0.9003,0.0105,0.0070
7,0.0772,0.0134,0.1157,0.9096,0.0092,0.0064
8,0.0793,0.0105,0.1022,0.9408,0.0081,0.0067
9,0.0819,0.0140,0.1185,0.8920,0.0094,0.0069


# Tuning the Ridge Model Further

In [ ]:
tuned_ridge = tune_model(ridge, optimize = 'RMSLE', n_iter= 20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0870,0.0164,0.1279,0.8823,0.0102,0.0074
1,0.0723,0.0091,0.0952,0.9519,0.0073,0.0060
2,0.0761,0.0110,0.1050,0.9418,0.0082,0.0064
3,0.0577,0.0055,0.0744,0.9631,0.0057,0.0048
4,0.0826,0.0140,0.1183,0.9266,0.0095,0.0070
5,0.0871,0.0156,0.1249,0.8959,0.0096,0.0073
6,0.0819,0.0176,0.1328,0.8981,0.0106,0.0069
7,0.0698,0.0101,0.1003,0.9320,0.0079,0.0058
8,0.0796,0.0111,0.1052,0.9373,0.0083,0.0068
9,0.0793,0.0131,0.1146,0.8989,0.0092,0.0067


# Finalizing the Tuned Ridge Model to the entire Train Set

In [ ]:
final_ridge = finalize_model(tuned_ridge)

# Creating Predictions on Test Set and Reverse Transforming the Predicted Target column

In [ ]:
pred = predict_model(final_ridge,data = test)
pred['SalePrice'] = np.exp(pred['Label'])

# Selecting the Necessary Columns and Saving to a CSV File

In [ ]:
final_pred = pred[['Id','SalePrice']]
final_pred.to_csv('kaggle_final_pred.csv')